In [1]:
#   !pip install selenium
#   !pip install webdriver-manager 

### Importar as bibliotecas

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

### Vai baixar o driver referente a versão do nosso Chrome instalado

In [3]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

In [4]:
url = 'https://www.fundamentus.com.br/resultado.php'
driver.get(url)

### Pegar a tabela do site fundamentus, e colocar no pandas

Passamos o thousand como . e decimal como , para não ter bug, pois nos EUA é ao contrário


In [5]:
local_tabela = '/html/body/div[1]/div[2]/table'

elemento = driver.find_element("xpath", local_tabela)

html_tabela = elemento.get_attribute("outerHTML")

tabela = pd.read_html(str(html_tabela), thousands = '.', decimal = ',')[0]

tabela

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,PORP4,2.40,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.0,2.239900e+07,0.00,"13,66%"
1,CFLU4,1000.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"8,88%","10,72%",1.10,"17,68%","32,15%",0.0,6.035100e+07,0.06,"8,14%"
2,CSTB4,147.69,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
3,CSTB3,150.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.0,8.420670e+09,0.14,"31,91%"
4,IVTT3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","-0,40%",0.0,1.083050e+09,0.00,"20,67%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981,PRBC4,14.54,502.29,40.26,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","8,02%",0.0,1.176410e+09,0.00,"-6,01%"
982,UBBR4,7.49,610.27,1.99,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
983,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"
984,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.0,1.031720e+10,0.00,"10,58%"


### Pegamos os indicadores fundalistas de todas as empresas da bolsa e agora vamos construir a tabela final 

1: Setamos papel como indice

2: Selecionamos os dados que precisamos

In [6]:
tabela = tabela.set_index('Papel')

In [7]:
tabela = tabela[['Cotação', 'EV/EBIT', 'ROIC', 'Liq.2meses']]
tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses
Papel,,,,
PORP4,2.40,0.0,"0,00%",0.0
CFLU4,1000.00,0.0,"17,68%",0.0
CSTB4,147.69,0.0,"22,40%",0.0
CSTB3,150.00,0.0,"22,40%",0.0
IVTT3,0.00,0.0,"0,00%",0.0
...,...,...,...,...
PRBC4,14.54,0.0,"0,00%",0.0
UBBR4,7.49,0.0,"0,00%",0.0
UBBR11,14.75,0.0,"0,00%",0.0


### Tratar os textos

In [8]:
tabela['ROIC'] = tabela['ROIC'].str.replace("%", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(".", "")
tabela['ROIC'] = tabela['ROIC'].str.replace(",", ".")
tabela['ROIC'] = tabela['ROIC'].astype(float)
tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses
Papel,,,,
PORP4,2.40,0.0,0.00,0.0
CFLU4,1000.00,0.0,17.68,0.0
CSTB4,147.69,0.0,22.40,0.0
CSTB3,150.00,0.0,22.40,0.0
IVTT3,0.00,0.0,0.00,0.0
...,...,...,...,...
PRBC4,14.54,0.0,0.00,0.0
UBBR4,7.49,0.0,0.00,0.0
UBBR11,14.75,0.0,0.00,0.0


### Fazendo filtros e criando o Rank

In [9]:
tabela = tabela[tabela['Liq.2meses'] > 1000000]

In [10]:
tabela = tabela[tabela['EV/EBIT'] > 0]
tabela = tabela[tabela['ROIC'] > 0]


In [11]:
tabela['ranking_ev_ebit'] = tabela['EV/EBIT'].rank(ascending = True)
tabela['ranking_roic'] = tabela['ROIC'].rank(ascending = False)
tabela['ranking_total'] = tabela['ranking_ev_ebit'] + tabela['ranking_roic']

tabela = tabela.sort_values('ranking_total')

tabela

,Cotação,EV/EBIT,ROIC,Liq.2meses,ranking_ev_ebit,ranking_roic,ranking_total
Papel,,,,,,,
PSSA3,25.26,0.20,60.76,3.378900e+07,1.0,1.0,2.0
PETR4,35.44,2.60,28.95,1.737520e+09,4.0,9.5,13.5
UNIP6,70.85,4.46,38.68,1.591610e+07,11.5,3.0,14.5
PETR3,38.48,2.75,28.95,4.785100e+08,5.0,9.5,14.5
KEPL3,10.50,4.64,43.61,1.328970e+07,16.5,2.0,18.5
...,...,...,...,...,...,...,...
PCAR3,3.37,42.33,0.24,3.933670e+07,181.0,189.0,370.0
CBAV3,3.68,81.13,0.53,1.638600e+07,184.0,186.0,370.0
AMAR3,2.70,114.65,0.58,3.333730e+06,187.0,185.0,372.0


In [17]:
melhores_acoes = tabela.head(10)

In [21]:
tickers = melhores_acoes.index
tickers


Index(['PSSA3', 'PETR4', 'UNIP6', 'PETR3', 'KEPL3', 'WIZC3', 'VLID3', 'FESA4',
       'PLPL3', 'GOAU4'],
      dtype='object', name='Papel')

# Vamos imoportar as bibliotecas para conectar e fazer operações

In [22]:
!pip install MetaTrader5

     ---------------------------------------- 0.0/49.9 kB ? eta -:--:--
     -------- ------------------------------- 10.2/49.9 kB ? eta -:--:--
     --------------- ---------------------- 20.5/49.9 kB 217.9 kB/s eta 0:00:01
     ------------------------------- ------ 41.0/49.9 kB 245.8 kB/s eta 0:00:01
     ------------------------------- ------ 41.0/49.9 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 49.9/49.9 kB 210.7 kB/s eta 0:00:00


In [23]:
import MetaTrader5 as mt5

### Inicializar o MetaTrader 

In [26]:
mt5.initialize()

False

### Configurar dicionário de ordem e compra das ações

In [30]:
for ticker in tickers:
    
    mt5.symbol_select(ticker) # Selecionamos as empresas
    preco = mt5.symbol_info(ticker).ask
    quantidade = 100.0 # tem que colocar o .0, inteiros não funcionam, tem que ser um float
    
    ordem_compra = {
    "action": mt5.TRADE_ACTION_DEAL, # Trade_action_deal = trade a mercado, igual tem que passar preço
    "symbol": ticker, 
    "volume": quantidade,
    "type": mt5.ORDER_TYPE_BUY,
    "price": preco,
    "magic": 1,
    "comment": "Trade Hacekando a Bolsa",
    "type_time": mt5.ORDER_TIME_DAY, # Envia apenas quando o mercado está aberto
    "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    
    compra = mt5.order_send(ordem.compra)
    print(compra)